In [1]:
import llama_cpp

llama_cpp.__version__

'0.3.9'

In [2]:
from llama_cpp import Llama

llm_chat = Llama(model_path = "./llama-2-7b.Q4_K_S.gguf", n_gpu_threads=-1, use_mmap=False, verbose=False, n_ctx=4096, n_threads=8)
llm_emb = Llama(model_path = "./llama-2-7b.Q4_K_S.gguf", n_gpu_threads=-1, use_mmap=False, verbose=False, n_ctx=4096, n_threads=8, embedding=True)

In [3]:
import os
txt_path = "../data/txt"
pdf_path = "../data/pdf"

pdf_files = [f for f in os.listdir(pdf_path) if os.path.isfile(os.path.join(pdf_path, f))]
full_txt_files = [os.path.join(txt_path, f) for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f))]

txt = [open(f, encoding="utf-8").read() for f in full_txt_files]
all_txt = "[NEW FILE] ".join(txt)

In [4]:
def chunk_with_llamacpp(text, chunk_size=512, overlap=64):
    b = text.encode("utf-8")
    toks = llm_emb.tokenize(b, add_bos=False, special=False)
    chunks = []
    for i in range(0, len(toks), chunk_size - overlap):
        window = toks[i : i + chunk_size]
        chunks.append(llm_emb.detokenize(window))
        if i + chunk_size >= len(toks):
            break

    return chunks

chunks = chunk_with_llamacpp(all_txt)
print(f"Generated {len(chunks)} chunks")

Generated 298 chunks


In [5]:
chunks_txt = [c.decode("utf-8", errors="ignore") if isinstance(c, (bytes, bytearray)) else c for c in chunks]

In [39]:
"""
with open("chunks_txt.pkl", "wb") as file:
    pickle.dump(chunks_txt, file)
"""

In [6]:
import numpy as np

res = llm_emb.create_embedding(chunks_txt[:2])
print(len(res["data"]))
print(np.array(res["data"][0]["embedding"]).shape)
print(np.array(res["data"][1]["embedding"]).shape)

2
(512, 4096)
(512, 4096)


In [ ]:
from tqdm import tqdm

emb_list = []
for chunk in tqdm(chunks_txt, desc="Embedding chunks"):
    resp = llm_emb.create_embedding(input=chunk)
    emb = resp["data"][0]["embedding"]
    emb_list.append(emb)

emb_array = np.array(emb_list, dtype=np.float32)
np.save("embeddings.npy", emb_array)
print(f"Saved {emb_array.shape[0]} embeddings (dim={emb_array.shape[1]}) to embeddings.npy")

In [25]:
import pickle
"""
with open("embeddings_list.pkl", "wb") as file:
    pickle.dump(emb_list, file)
"""

In [26]:
with open('embeddings_list.pkl', 'rb') as file:
    emb_list = pickle.load(file)

298

In [ ]:
emb_list[0]

In [ ]:
n_chunks = len(emb_list)
dim = 4096
fixed_length = 512
pooled_np = []

for tok_emb in emb_list:
    n, d = len(tok_emb), len(tok_emb[0])
    if n < fixed_length:
        pad = np.zeros((fixed_length - n, d), dtype=np.float32)
        tok_emb = np.vstack([tok_emb, pad])
    elif n > fixed_length:
        tok_emb = tok_emb[:fixed_length]
    pooled_np.append(np.array(tok_emb,dtype=np.float32).mean(axis=0))

# 3) stack into one array of shape (n_chunks, 512, dim)
pooled_np = np.stack(pooled_np, axis=0)
print("pooled_np.shape =", pooled_np.shape)  # → (298, 512, 4096)

"""
np.save("pooled_np.npy", pooled_np)
print("Saved raw token embeddings to pooled_np.npy")
"""

In [38]:
pooled_np = np.load("pooled_np.npy")

In [ ]:
import faiss
import pickle
import numpy as np

pooled_np = np.load("pooled_np.npy")
with open("chunks.pkl", "rb") as f:
    chunks_txt = pickle.load(f)

n_chunks, dim = pooled_np.shape
index = faiss.IndexFlatL2(dim)
index.add(pooled_np.astype(np.float32)) 

print(f"✔️  Indexed {index.ntotal} chunk embeddings (dim={dim})")
faiss.write_index(index, "rag_index.faiss")

✔️  Indexed 298 chunk embeddings (dim=4096)


In [ ]:
import pickle
import numpy as np
import faiss
from llama_cpp import Llama


index = faiss.read_index("rag_index.faiss")
with open("chunks.pkl", "rb") as f:
    chunks = pickle.load(f)   # List[str]


SYSTEM = (
    "You are in charge of advising a finance company on information related to Uber. "
    "Justify any claims with examples from the context provided. "
    "Maintain a serious and professional tone."
)

EXAMPLE = """
Example:
Context:
Uber’s rideshare revenue grew 20% last quarter.

Question:
What does this growth imply for Uber’s position in the market?
Answer:
This 20% rideshare revenue increase underscores Uber’s ability to capture market share from traditional taxis, "
which positions it as the dominant on-demand mobility provider. [source: snippet 0]
"""


def retrieve(query: str, k: int = 5) -> list[str]:
    resp = llm_emb.create_embedding(input=[query])
    tok_emb = np.array(resp["data"][0]["embedding"], dtype=np.float32)
    
    emb = tok_emb.mean(axis=0)
    q_emb = emb.reshape(1, -1)  

    _, I = index.search(q_emb, k)
    return [chunks[i] for i in I[0]]


def answer(query: str) -> str:
    docs = retrieve(query, k=5)

    numbered = [f"[snippet {i}]\n{txt}" for i, txt in enumerate(docs)]
    context = "\n\n---\n\n".join(numbered)

    prompt = f"""{SYSTEM}

{EXAMPLE}

Context:
{context}

Question:
{query}

Answer:"""

    out = llm_chat(prompt, max_tokens=256, temperature=0.1)
    return out["choices"][0]["text"].strip()

if __name__ == "__main__":
    print("🔍 Query RAG (blank to quit)")
    while True:
        q = input("Query> ").strip()
        if not q:
            break
        print("🤖", answer(q), "\n")


🔍 Query RAG (blank to quit)


Query>  Explain what uber's next goal is?


🤖 I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the business. I think the next goal is to continue to grow the busin

KeyboardInterrupt: Interrupted by user

In [ ]:
import pickle
import numpy as np
import faiss
from llama_cpp import Llama

# ── CONFIG ────────────────────────────────────────────────────────────────────
MAX_CTX           = 2048
MAX_GEN           = 128
RESERVED          = MAX_GEN + 50
MAX_PROMPT_TOKENS = MAX_CTX - RESERVED

SYSTEM = (
    "You are a concise, professional assistant advising a finance company about Uber. "
    "Use only the provided context; justify claims with snippet citations."
)

"""
# ── MODEL INITS ───────────────────────────────────────────────────────────────
llm_emb = Llama(
    model_path="path/to/model.gguf",
    gpu_layers=-1,       # all on GPU
    n_ctx=MAX_CTX,
    n_threads=8,
    use_mmap=False,
    use_mlock=True
)
llm_chat = Llama(
    model_path="path/to/model.gguf",
    gpu_layers=-1,
    n_ctx=MAX_CTX,
    n_threads=8,
    use_mmap=False,
    use_mlock=True
)
"""

# ── LOAD INDEX ────────────────────────────────────────────────────────────────
index = faiss.read_index("rag_index.faiss")
with open("chunks.pkl","rb") as f:
    chunks = pickle.load(f)

# ── RETRIEVAL ──────────────────────────────────────────────────────────────────
def retrieve(query: str, k: int = 3) -> list[str]:
    resp    = llm_emb.create_embedding(input=[query])
    tok_emb = np.array(resp["data"][0]["embedding"], dtype=np.float32)
    emb     = tok_emb.mean(axis=0).reshape(1, -1)   # (1,4096)
    _, I    = index.search(emb, k)
    return [chunks[i] for i in I[0]]

# ── PROMPT BUILDER ────────────────────────────────────────────────────────────
def build_prompt(query: str, docs: list[str]) -> str:
    numbered = [f"[snippet {i}]\n{txt}" for i, txt in enumerate(docs)]
    while True:
        context = "\n\n---\n\n".join(numbered)
        prompt  = (
            SYSTEM + "\n\n"
            f"Context:\n{context}\n\n"
            f"Question:\n{query}\n\n"
            "Answer:"
        )
        if len(prompt.split()) <= MAX_PROMPT_TOKENS or not numbered:
            return prompt
        numbered.pop()  # drop last snippet to shrink context

# ── ANSWERING (streaming) ──────────────────────────────────────────────────────
def answer_stream(query: str):
    docs   = retrieve(query, k=3)
    prompt = build_prompt(query, docs)

    # NOTE: no stop=["\n"] here — let the model actually write!
    for token in llm_chat(
        prompt,
        max_tokens=MAX_GEN,
        temperature=0.2,
        top_p=0.9,
        repeat_penalty=1.2,
        stream=True
    ):
        print(token["choices"][0]["text"], end="", flush=True)
    print()  # final newline

# ── REPL ─────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    print("🔍 Query RAG (blank to quit)")
    while True:
        q = input("Query> ").strip()
        if not q:
            break
        answer_stream(q)


🔍 Query RAG (blank to quit)


Query>  what is uber spending their money on?



Uber is spending their money on the following:

1. Uber Eats (food delivery)
2. Rideshare business in the US and abroad
3. Self-driving cars
4. Other miscellaneous projects like bike sharing, etc.

---


Query>  What possible risks are there if we merge with uber?



There are a few potential risks that could arise if we merge with Uber. First, there is the risk of losing our unique identity and culture as an organization. If we were to combine forces with another company, it would be easy for us to lose sight of what makes us special and instead focus on becoming just like everyone else. Secondly, there's always a chance that something goes wrong during integration or implementation which could lead to cost overruns or delays in getting products out the door. Finally, if we were able to merge successfully then it would mean less competition within our industry - meaning fewer opportunities for


In [55]:
llm_chat("How are you doing?", max_tokens=128, temperature=0.1)["choices"][0]["text"].strip()

"I hope you are doing well. everybody. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well. I'm doing well"